# Qualitative study on Fermi estimation problem

In [ ]:
%pip install openai
%pip install langchain
%pip install python-dotenv

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

## Task breakdown - formalise estimation into sub steps

human-in-the-loop as the invigilator (monitor model output and decide what's sensible as input)

In [11]:
# Freddie suggestion: how can a language model generalise the model
# such as estimating the number of nursery in Beijing?


# STEP 0 - natural language understanding
# What do I want to estimate here
# rephrase: what the question is asking me about?

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
## set up LLM
llm = OpenAI(temperature=0.9, max_tokens=1000)

## set up prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="""\
Assistant will not directly answer.\ Assistant wants to clarify what it means.
Assistant will rephrase the content after QUESTION \
QUESTION: {question}""",
)

## create a chain (or, chain the action)
chain = LLMChain(llm=llm, prompt=prompt)
with get_openai_callback() as cb:
    output = chain.run("How could I know how many nurseries are there in London?").strip()
    print(cb)

print(output)

Tokens Used: 54
	Prompt Tokens: 41
	Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.0010800000000000002
What is the number of nurseries located in London?


In [12]:
## intermediate task: categorize the LM output into 3 categories
prompt_format_output = PromptTemplate(
    input_variables=["LM_output"],
    template="""\
Assistant is ready to help user categorize LM output into 3 categorizes:
Assumptions, Calculation, and Final Answer.\
Assistant will only arrange the information into categorises. Assitant will not make up any new information not presented from LM output.\
LM output: {LM_output}""",
)

LM_output = """To estimate the number of days it takes for your red cell count to return to normal after donating a pint of blood, we can make some assumptions and calculations.

Assumptions:
1. A pint of blood is approximately 500 mL.
2. The average adult has about 5 liters (5,000 mL) of blood.
3. The average red blood cell (RBC) lifespan is about 120 days.
4. The body produces RBCs at a constant rate to maintain a stable RBC count.

Step 1: Calculate the percentage of blood donated
(500 mL donated) / (5,000 mL total blood) = 0.1 or 10%

Step 2: Estimate the percentage of RBCs lost
Since you donated 10% of your blood, we can assume you also lost about 10% of your RBCs.

Step 3: Calculate the time it takes to replace the lost RBCs
Since the average RBC lifespan is 120 days, we can assume that your body replaces about 1/120th of your RBCs every day. To replace the 10% of RBCs lost, we can use the following calculation:

(10% RBCs lost) / (1/120 RBCs replaced per day) = 12 days

So, it will take approximately 12 days for your red cell count to return to normal after donating a pint of blood.

REASON: We made assumptions about the volume of blood donated, the total blood volume, and the RBC replacement rate to estimate the time it takes for the red cell count to return to normal.

ESTIMATED VALUE: 10^1 (since 12 is in the order of magnitude of 10)"""

## create a chain (or, chain the action)
chain_format_output = LLMChain(llm=llm, prompt=prompt_format_output)
with get_openai_callback() as cb:
    output = chain_format_output.run(LM_output).strip()
    print(cb)

print(output)

Tokens Used: 706
	Prompt Tokens: 425
	Completion Tokens: 281
Successful Requests: 1
Total Cost (USD): $0.01412
Categorization:
Assumptions:
1. A pint of blood is approximately 500 mL.
2. The average adult has about 5 liters (5,000 mL) of blood.
3. The average red blood cell (RBC) lifespan is about 120 days.
4. The body produces RBCs at a constant rate to maintain a stable RBC count.

Calculations:
Step 1: Calculate the percentage of blood donated
(500 mL donated) / (5,000 mL total blood) = 0.1 or 10%

Step 2: Estimate the percentage of RBCs lost
Since you donated 10% of your blood, we can assume you also lost about 10% of your RBCs.

Step 3: Calculate the time it takes to replace the lost RBCs
Since the average RBC lifespan is 120 days, we can assume that your body replaces about 1/120th of your RBCs every day. To replace the 10% of RBCs lost, we can use the following calculation:

(10% RBCs lost) / (1/120 RBCs replaced per day) = 12 days

Final Answer: So, it will take approximately 1

In [ ]:
# intermediate task: evaluate a list of assumptions (list)
prompt_evaluate_assumption = PromptTemplate(
    input_variables=["question","LM_output"],
    template="""\
Assistant will assign a confidence score to a list of statements.\
Assistant will only \
assumption statements:\
 {LM_output}""",
)

LM_output = """To estimate the number of days it takes for your red cell count to return to normal after donating a pint of blood, we can make some assumptions and calculations.

Assumptions:
1. A pint of blood is approximately 500 mL.
2. The average adult has about 5 liters (5,000 mL) of blood.
3. The average red blood cell (RBC) lifespan is about 120 days.
4. The body produces RBCs at a constant rate to maintain a stable RBC count.
"""

## create a chain (or, chain the action)
chain_format_output = LLMChain(llm=llm, prompt=prompt_format_output)
with get_openai_callback() as cb:
    output = chain_format_output.run(LM_output).strip()
    print(cb)

print(output)